<a href="https://colab.research.google.com/github/rakhimovv/cv-course-hse-spring20/blob/master/03-edgescorners/edges_and_HOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# План
1. Границы на котике
2. Дескрипторы (HOG) на цифрах

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
image = cv2.imread("maxresdefault.jpg")

In [ ]:
image = (image[:, :, ::-1] / 255.).astype(np.float32)

In [ ]:
plt.imshow(image)

In [ ]:
gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=1)
gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=1)

In [ ]:
magnitude = np.sqrt(gx ** 2 + gy ** 2)

In [ ]:
plt.figure(figsize=(10, 7))
plt.imshow(magnitude)

In [ ]:
plt.imshow(np.abs(gx) * 2)

In [ ]:
plt.imshow(np.abs(gy) * 2)

In [ ]:
angle = np.arctan2(gy, gx)

In [ ]:
plt.hist(angle.reshape(-1), weights=magnitude.reshape(-1), bins=30);

In [ ]:
angle = angle.reshape(-1)

In [ ]:
angle[angle < 0] += np.pi

In [ ]:
plt.hist(angle, weights=magnitude.reshape(-1), bins=30);

# HOG classifier

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
digits = load_digits()

In [ ]:
plt.imshow(digits['images'][2], cmap='gray')

In [ ]:
digits['images'][2].shape

In [ ]:
BINS = 16

In [ ]:
def compute_hog(digit, bins=20):
    gx = cv2.Sobel(digit, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(digit, cv2.CV_32F, 0, 1, ksize=1)
    
    magnitude = np.sqrt(gx ** 2 + gy ** 2)
    angle = np.arctan2(gy, gx)
    angle[angle < 0] += np.pi
    
    hog, _ = np.histogram(angle, bins=bins, weights=magnitude, density=True)
    return hog

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(digits['images'], digits['target'], test_size=0.2)

In [ ]:
centroids = np.zeros((10, BINS))
images_in_class = [0] * 10

In [ ]:
for image, label in zip(X_train, y_train):
    centroids[label] += compute_hog((image / 16.).astype(np.float32), bins=BINS)
    images_in_class[label] += 1

In [ ]:
centroids = centroids / np.array(images_in_class)[:, None]

In [ ]:
predictions = []

In [ ]:
for image in X_test:
    hog = compute_hog((image / 16.).astype(np.float32), BINS)
    distance_to_centroid = np.linalg.norm(centroids - hog, axis=1)
    predictions.append(np.argmin(distance_to_centroid))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(predictions, y_test)